In [2]:
#Importando as bibliotecas necessárias
import os
import zipfile
from dotenv import load_dotenv
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import pyarrow as pv
import fastparquet as pq
import dask.dataframe as dd
import polars as pl

c:\Users\rodri\personal-projects\spotify-project\venv\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# Carrega variáveis de ambiente do arquivo .env
load_dotenv()

# Inicializa a API do Kaggle
api = KaggleApi()
api.authenticate()

# Especifica o dataset a ser baixado. Use o nome do dataset no formato 'autor/dataset'
dataset = 'sunnykakar/spotify-charts-all-audio-data'  # Exemplo de um dataset específico
data_path = os.getenv('DATA_CSV_RAW_PATH')  # Obtém o caminho para a pasta onde os dados serão armazenados

# Baixa o dataset para a pasta especificada
api.dataset_download_files(dataset, path=data_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/sunnykakar/spotify-charts-all-audio-data


In [3]:
load_dotenv()

csv_path = os.getenv('CSV_RAW_PATH')
parquet_path = os.getenv('PARQUET_RAW_PATH')
data_folder = os.getenv('RAW_DATA_FOLDER')

In [4]:
#funcao para obter informações de dados tipo csv ou parquet
def get_data_info(data_path, data_type):

    # Obten o tamanho do arquivo em MB
    file_size_mb = os.path.getsize(data_path) / (1024 * 1024)
    print(f"Tamanho do arquivo: {file_size_mb:.2f} MB")
    
    if data_type == 'csv': ddf = dd.read_csv(data_path)
    elif data_type == 'parquet': ddf = dd.read_parquet(data_path)
    else: print('Formato inválido')

    # Obter o número de linhas e colunas
    num_rows = ddf.shape[0].compute()
    num_columns = ddf.shape[1]

    print(f"Quantidade de linhas: {num_rows}")
    print(f"Quantidade de colunas: {num_columns}")
    

In [13]:
get_data_info(csv_path, 'csv')

Tamanho do arquivo: 25848.48 MB
Quantidade de linhas: 26174269
Quantidade de colunas: 29


In [7]:
#Transformando o aquivo de csv para parquet

def get_csv_to_parquet(csv_path, parquet_path, chunksize=10**6):
    # Lê o CSV em chunks de 1 milhão de linhas por vez
    chunks = pd.read_csv(csv_path, chunksize=chunksize, low_memory=True)
    
    # Itera sobre os chunks e escreve no formato Parquet
    for i, chunk in enumerate(chunks):
        # Escreve o primeiro chunk criando o arquivo
        if i == 0:
            chunk.to_parquet(parquet_path, engine='pyarrow', index=False)
        else:
            # Os chunks subsequentes são anexados ao arquivo Parquet existente
            chunk.to_parquet(parquet_path, engine='pyarrow', index=False, append=True)
    
    print("Transformação Concluída")